In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import os
import pandas as pd
import xarray as xr

In [2]:
# longitude range setting
# from -180 to 180
def format(longitude):
    if longitude > 180:
        longitude = longitude - 360
    else:
        longitude = longitude
    return longitude

In [3]:
rootdir = '/home/cz2397/data/cchdo-ctd/'

In [4]:
cchdo_ctd = MongoClient().eddies.cchdo_ctd

In [5]:
for subdir, dirs, files in tqdm(os.walk(rootdir)):
    for file in files:
        if file.endswith('ctd.nc'):
            path = os.path.join(subdir, file)
            ds = xr.open_dataset(path)
            cast = {
                'path': path,
                'time': pd.to_datetime(ds.time.values[0]),
                'location': {'type': 'Point',
                             'coordinates': [format(float(ds.longitude.values[0])), float(ds.latitude.values[0])],
                },
                'expocode': ds.EXPOCODE,
                'station': ds.STATION_NUMBER,
                'cast': ds.CAST_NUMBER,
                'category': 'CTD'
            }
            cchdo_ctd.insert_one(dict(cast))

739it [05:28,  1.54it/s]
